Lab 6 Nebraska

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [3]:
import psycopg2

conn = psycopg2.connect(host=get_secret('postgresql')['host'],
                      user=get_secret('postgresql')['username'],
                      password = get_secret('postgresql')['password'],
                      dbname=get_secret('postgresql')['engine'])

cur = conn.cursor()

## Q1- Population Growth Rate

Option 1

In [5]:
ql_sql = """
SELECT 
    p.year,
    p.pop AS population,
    ROUND(
        ( (p.pop::numeric - LAG(p.pop) OVER (ORDER BY p.year)) 
          / LAG(p.pop) OVER (ORDER BY p.year) ) * 100, 
        2
    ) AS growth_rate_percent
FROM 
    population p
JOIN 
    name n 
    ON p.fips = n.fips
WHERE 
    n.name = 'Nebraska'
ORDER BY 
    p.year;
"""

In [15]:
import pandas as pd
ql_df = pd.read_sql(ql_sql, con=conn)
ql_df.head()

/tmp/ipykernel_18850/2819096148.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ql_df = pd.read_sql(ql_sql, con=conn)


,year,population,growth_rate_percent
0,2005,1706976,NaN
1,2006,1768331,3.59
2,2007,1774571,0.35
3,2008,1783432,0.50
4,2009,1796622,0.74


In [25]:
ql_df.plot(x='year', y='income')

KeyError: 'income'

In [26]:
! pip install jupyter-ai-magics[all] -q

In [27]:
import warnings
warnings.simplefilter('ignore')

In [28]:
import os
os.environ["OPENAI_API_KEY"] = get_secret('openai')['api_key']

KeyError: 'api_key'

In [29]:
%load_ext jupyter_ai_magics

The jupyter_ai_magics extension is already loaded. To reload it, use:
  %reload_ext jupyter_ai_magics


In [31]:
%%ai gpt4
Calculate the growth rate with the data provided in {va_df}, use the va_df directly, and visualize the result on a line chart

OSError: Authentication environment variable OPENAI_API_KEY is not set.
An authentication token is required to use models from the OpenAI provider.
Please specify it via `%env OPENAI_API_KEY=token`. 